# HW6 FX Carry 
Bailey Meche

## Submission notes  

This notebook implements the HW6 specification exactly:
- Weekly W-WED entry/exit; FX/OIS aligned within ±2 days.
- Borrow in GBP at OIS+50bp on 4/5 notional (5x leverage); lend via a 5Y par bond in each EM currency.
- Coupon fixed at entry 5Y swap rate; quarterly coupons; mark-to-market one week later using the new swap curve bootstrapped to zero-coupon discounts.
- Trades only when the lending 5Y swap exceeds the funding 5Y swap by at least 50bp.
- All cashflows converted to USD.


#  Report

## Spec implemented  
- Weekly entry/exit on W-WED; nearest trading day within ±2 days for FX/OIS.
- Funding GBP: borrow at OIS+50bp on 4/5 notional (5x leverage).
- Lending: buy 5Y par bond, coupon fixed at entry 5Y swap rate, quarterly coupons.
- Filter: trade if (lend 5Y swap − fund 5Y swap) ≥ 50bp.
- MTM: reprice remaining CFs one week later on new curve (bootstrapped ZC from par curve).
- All flows in USD; returns shown as equity return (P&L / $2MM) and notional-normalized.

## Data + interpolation audit
- Curves densified to business days via time interpolation (max gap 45 bdays).
- Weekly curves sampled within ±7 days.
- Enforced: curve must cover **1Y and 5Y** at entry and exit.
- See: `curve_time_interpolation_audit.csv`, `alignment_staleness_summary.csv`, `entry_par_check.csv`.

## Performance
### Variable-capital portfolio (avg across active positions)
| n_weeks | mean_w | vol_w | sharpe_w | mean_ann | vol_ann | sharpe_ann | terminal_wealth | max_drawdown |
|---|---|---|---|---|---|---|---|---|
| 365 | -0.006287 | 0.118949 | -0.052851 | -0.326902 | 0.857751 | -0.381115 | 0.006521 | -0.998644 |

### Fixed-allocation portfolio (inactive=0)
| n_weeks | mean_w | vol_w | sharpe_w | mean_ann | vol_ann | sharpe_ann | terminal_wealth | max_drawdown |
|---|---|---|---|---|---|---|---|---|
| 365 | -0.001835 | 0.044655 | -0.041098 | -0.095433 | 0.322014 | -0.296365 | 0.355795 | -0.842076 |

## Correlations, risk factors, contributions
- Correlations: `currency_corr_all_weeks.csv` and `currency_corr_active_overlap.csv`
- Factor regression (HAC): `market_factor_regs.csv`
- Sharpe variance shares: `sharpe_variance_contrib_fixed.csv`
- Worst-10-week drawdown contributors: `drawdown_contrib_worst10w.csv`
![alt text](wealth_fixed.png)![alt text](drawdown_fixed.png)![alt text](cum_pnl_by_ccy.png)![alt text](corr_heatmap_all.png)
- `figures/wealth_fixed.png`
- `figures/drawdown_fixed.png`
- `figures/cum_pnl_by_ccy.png`
- `figures/corr_heatmap_all.png`


In [ ]:
from pathlib import Path
from datetime import datetime, timezone
import json, os, re, subprocess, platform
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import statsmodels.api as sm

BASE=Path('.').resolve()
OUT=BASE/'outputs'
FIG=OUT/'figures'
DATA=BASE/'data'
DATA_CLEAN=BASE/'data_clean'
OUT.mkdir(parents=True,exist_ok=True)
FIG.mkdir(parents=True,exist_ok=True)

def backup_if_exists(path: Path):
    if path.exists():
        ts=datetime.now().strftime('%Y%m%d_%H%M%S')
        bak=path.with_name(path.name+f'.bak_{ts}')
        path.rename(bak)
        print('Backed up existing',path,'->',bak)

def write_text_new(path: Path, text: str):
    backup_if_exists(path)
    path.write_text(text)

def write_csv_new(df: pd.DataFrame, path: Path, index=False):
    backup_if_exists(path)
    df.to_csv(path, index=index)

manifest={
 'timestamp_utc':datetime.now(timezone.utc).isoformat(),
 'python':platform.python_version(),'pandas':pd.__version__,'numpy':np.__version__,
 'files_chosen':{},'coverage':{},'weekly_obs':None,'n_currencies':None
}
try:
    manifest['git_commit']=subprocess.check_output(['git','rev-parse','--short','HEAD'],text=True).strip()
except Exception:
    manifest['git_commit']='unknown'
print(manifest)

{'timestamp_utc': '2026-02-18T05:21:06.987181+00:00', 'python': '3.13.9', 'pandas': '2.3.3', 'numpy': '2.3.3', 'files_chosen': {}, 'coverage': {}, 'weekly_obs': None, 'n_currencies': None, 'git_commit': '84c6f27'}


In [12]:
# ============================================
# LOAD + CANONICALIZE INPUTS (single source of truth)
# Produces:
#   ois_on  : pd.Series (daily, decimal), UK overnight OIS (IUDSOIA)
#   fx_usd_per_ccy : pd.DataFrame (daily), columns include EM + GBP, values USD per 1 CCY
#   curves_long : pd.DataFrame columns [date, ccy, tenor_y, par_swap] (decimal)
#   gbp_s5 : pd.Series (daily, decimal), GBP 5Y par swap proxy used in filter
# ============================================
# ---- EM curves (Bloomberg exports; robust parser) ----
import numpy as np
import pandas as pd
import re
from pathlib import Path

EM = ["BRL","NGN","PKR","TRY","ZAR"]

EM_CURVE_DIRS = [DATA, DATA_CLEAN, BASE / "_data", BASE / "data_raw"]
EM_CURVE_GLOBS = [
    "*Emerging*Mkt*YC*.csv",
    "*Emerging*YC*.csv",
    "*EM*YC*.csv",
    "*YC*Emerging*.csv",
]

def _read_csv_robust(fp: Path) -> pd.DataFrame:
    # Try common Bloomberg layouts: single header, two-row header, messy preamble
    try:
        df = pd.read_csv(fp)
        if df.shape[1] >= 2:
            return df
    except Exception:
        pass

    # Two-row header (e.g., security row + PX_LAST row)
    try:
        df2 = pd.read_csv(fp, header=[0, 1])
        if isinstance(df2.columns, pd.MultiIndex) and df2.shape[1] >= 2:
            # Collapse to first level (security names)
            df2.columns = [str(c[0]) for c in df2.columns]
            return df2
    except Exception:
        pass

    # Fallback: python engine + skip bad lines
    df = pd.read_csv(fp, engine="python", on_bad_lines="skip")
    return df

def _detect_header_row(fp: Path, max_rows: int = 30) -> int | None:
    # Finds a row containing "date" / "dates" to use as header
    raw = pd.read_csv(fp, header=None, nrows=max_rows, engine="python", on_bad_lines="skip")
    for i in range(min(max_rows, len(raw))):
        row = raw.iloc[i].astype(str).str.strip().str.lower()
        if row.isin(["date", "dates", "as of date", "asof", "dt"]).any():
            return i
    return None

def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    # drop fully empty cols
    df = df.dropna(axis=1, how="all")
    return df

def _find_date_col(df: pd.DataFrame) -> str | None:
    cols = list(df.columns)
    # Prefer explicit date columns
    for c in cols:
        if re.search(r"\b(date|dates|asof|as of)\b", str(c), flags=re.I):
            return c
    # Else try first col if it parses well
    c0 = cols[0] if cols else None
    if c0 is None:
        return None
    dt = pd.to_datetime(df[c0], errors="coerce")
    if dt.notna().mean() > 0.7:
        return c0
    return None

def _strip_px_last_row(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    # Bloomberg sometimes puts a first “data” row with PX_LAST repeated
    if df.shape[0] == 0:
        return df
    first = df.iloc[0].copy()
    # ignore date cell
    vals = first.drop(labels=[date_col], errors="ignore").astype(str).str.upper().str.strip()
    if vals.size and (vals.isin(["PX_LAST","PX_MID","LAST_PRICE","LAST"]).mean() > 0.8):
        return df.iloc[1:].copy()
    return df

def _infer_ccy_tenor_from_string(s: str):
    s0 = str(s).upper()

    # Preferred: GTBRL 5Y, GTBRL5Y, GTBRL 60M, etc.
    m = re.search(r"GT\s*([A-Z]{3})\s*([0-9]+(?:\.[0-9]+)?)\s*(Y|YR|M|W|D)\b", s0)
    if not m:
        m = re.search(r"\bGT([A-Z]{3})\s*([0-9]+(?:\.[0-9]+)?)\s*(Y|YR|M|W|D)\b", s0)
    if m:
        ccy = m.group(1)
        n = float(m.group(2))
        u = m.group(3)
        tenor_y = n if u in {"Y","YR"} else (n/12.0 if u=="M" else (n/52.0 if u=="W" else n/365.0))
        return ccy, tenor_y

    # Alternative: contains CCY and tenor like "BRL 5Y", "TRY5Y"
    m = re.search(r"\b(" + "|".join(EM) + r")\b.*?([0-9]+(?:\.[0-9]+)?)\s*(Y|YR|M|W|D)\b", s0)
    if m:
        ccy = m.group(1)
        n = float(m.group(2))
        u = m.group(3)
        tenor_y = n if u in {"Y","YR"} else (n/12.0 if u=="M" else (n/52.0 if u=="W" else n/365.0))
        return ccy, tenor_y

    # Another alternative: tenor like "5Y" but CCY maybe in filename later
    m = re.search(r"\b([0-9]+(?:\.[0-9]+)?)\s*(Y|YR|M|W|D)\b", s0)
    if m:
        n = float(m.group(1))
        u = m.group(2)
        tenor_y = n if u in {"Y","YR"} else (n/12.0 if u=="M" else (n/52.0 if u=="W" else n/365.0))
        return None, tenor_y

    return None, np.nan

def _parse_em_curve_file(fp: Path) -> pd.DataFrame:
    import numpy as np
    import pandas as pd
    import re

    EM = ["BRL","NGN","PKR","TRY","ZAR"]

    def _parse_name(name: str):
        s = str(name).upper()
        m = re.search(r"\bGT\s*([A-Z]{3})\s*([0-9]+(?:\.[0-9]+)?)\s*(Y|YR|M|W|D)\b", s)
        if not m:
            m = re.search(r"\bGT([A-Z]{3})\s*([0-9]+(?:\.[0-9]+)?)\s*(Y|YR|M|W|D)\b", s)
        if not m:
            return None, np.nan
        ccy = m.group(1)
        n = float(m.group(2))
        u = m.group(3)
        tenor_y = n if u in {"Y","YR"} else (n/12.0 if u=="M" else (n/52.0 if u=="W" else n/365.0))
        return ccy, tenor_y

    def _build_long(dates, values, ccy, tenor_y):
        out = pd.DataFrame({"date": dates, "ccy": ccy, "tenor_y": float(tenor_y), "par_swap": values})
        out["date"] = pd.to_datetime(out["date"], errors="coerce")
        out["par_swap"] = pd.to_numeric(out["par_swap"], errors="coerce")
        out = out.dropna(subset=["date","par_swap"])
        out["date"] = out["date"].dt.tz_localize(None)
        return out

    # ---------- Attempt 1: header-based paired columns ----------
    try:
        df = pd.read_csv(fp)
        cols = list(df.columns)
        gt_idx = [i for i, c in enumerate(cols)
                  if re.search(r"\bGT[A-Z]{3}\s*[0-9]+(?:\.[0-9]+)?\s*(Y|YR|M|W|D)\b", str(c).upper())]
        out = []
        for i in gt_idx:
            if i + 1 >= len(cols):
                continue
            ccy, ten = _parse_name(cols[i])
            if ccy not in EM or not np.isfinite(ten):
                continue
            dates = df.iloc[:, i]
            vals = df.iloc[:, i+1]
            out.append(_build_long(dates, vals, ccy, ten))
        if out:
            return pd.concat(out, ignore_index=True)
    except Exception:
        pass

    # ---------- Attempt 2: header=None, first row contains series names ----------
    raw = pd.read_csv(fp, header=None, engine="python", on_bad_lines="skip")
    if raw.shape[0] < 2 or raw.shape[1] < 2:
        return pd.DataFrame(columns=["date","ccy","tenor_y","par_swap"])

    names = raw.iloc[0].astype(str)
    out = []
    used = set()

    for j in range(raw.shape[1] - 1):
        if j in used:
            continue
        name = names.iloc[j]
        if not re.search(r"\bGT[A-Z]{3}\s*[0-9]+(?:\.[0-9]+)?\s*(Y|YR|M|W|D)\b", name.upper()):
            continue

        ccy, ten = _parse_name(name)
        if ccy not in EM or not np.isfinite(ten):
            continue

        dates = raw.iloc[1:, j]
        vals = raw.iloc[1:, j+1]
        out.append(_build_long(dates, vals, ccy, ten))

        used.add(j)
        used.add(j+1)

    if out:
        return pd.concat(out, ignore_index=True)

    return pd.DataFrame(columns=["date","ccy","tenor_y","par_swap"])

# Collect files across likely directories
em_files = []
for d in EM_CURVE_DIRS:
    if d.exists():
        for g in EM_CURVE_GLOBS:
            em_files += list(d.glob(g))
em_files = sorted({p.resolve() for p in em_files if p.is_file()}, key=lambda p: (p.stat().st_mtime, p.name), reverse=True)

if not em_files:
    raise FileNotFoundError(f"No EM curve CSVs found. Searched dirs={EM_CURVE_DIRS} globs={EM_CURVE_GLOBS}")

parts = []
failed = []
for fp in em_files:
    tmp = _parse_em_curve_file(fp)
    if tmp is None or tmp.empty:
        failed.append(fp)
        continue
    parts.append(tmp)

if not parts:
    # Print one file sample to make debugging deterministic
    samp = failed[0]
    raw_preview = pd.read_csv(samp, header=None, nrows=12, engine="python", on_bad_lines="skip")
    raise RuntimeError(
        "Failed to parse any EM curve files.\n"
        f"First file tried: {samp}\n"
        f"Preview (first 12 rows):\n{raw_preview}"
    )

curves_long = pd.concat(parts, ignore_index=True)
curves_long["date"] = pd.to_datetime(curves_long["date"], errors="coerce").dt.tz_localize(None)
curves_long["ccy"] = curves_long["ccy"].astype(str).str.upper().str.strip()
curves_long["par_swap"] = pd.to_numeric(curves_long["par_swap"], errors="coerce")
curves_long = curves_long.dropna(subset=["date","ccy","tenor_y","par_swap"])
curves_long = curves_long[curves_long["ccy"].isin(EM)]
curves_long = curves_long[(curves_long["tenor_y"] > 0) & (curves_long["tenor_y"] <= 30)]
if curves_long["par_swap"].median() > 1.0:
    curves_long["par_swap"] = curves_long["par_swap"] / 100.0
curves_long = curves_long.groupby(["date","ccy","tenor_y"], as_index=False)["par_swap"].mean()

print("Parsed EM curves:",
      "| files_used", len(parts),
      "| rows", len(curves_long),
      "| ccy_counts", curves_long["ccy"].value_counts().to_dict(),
      "| tenor_range", (float(curves_long["tenor_y"].min()), float(curves_long["tenor_y"].max())))


Parsed EM curves: | files_used 5 | rows 9792 | ccy_counts {'BRL': 3973, 'TRY': 2890, 'ZAR': 2317, 'PKR': 349, 'NGN': 263} | tenor_range (1.0, 10.0)


In [ ]:
def _tenor_to_years(x):
    if x is None or (isinstance(x, float) and np.isnan(x)): 
        return np.nan
    s = str(x).strip().upper()
    m = re.match(r"^(\d+(\.\d+)?)(Y|YR|YEARS?)$", s)
    if m: return float(m.group(1))
    m = re.match(r"^(\d+(\.\d+)?)(M|MO|MONTHS?)$", s)
    if m: return float(m.group(1))/12.0
    m = re.match(r"^(\d+(\.\d+)?)(W|WK|WEEKS?)$", s)
    if m: return float(m.group(1))/52.0
    m = re.match(r"^(\d+(\.\d+)?)(D|DAY|DAYS?)$", s)
    if m: return float(m.group(1))/365.0
    try: 
        return float(s)
    except Exception:
        return np.nan

def boe_to_long(boe_df: pd.DataFrame) -> pd.DataFrame:
    b = boe_df.copy()

    # Try identify date col
    date_col = next((c for c in b.columns if str(c).lower() in {"date","dt","asof","obs_date","observation_date"}), None)
    if date_col is not None:
        b[date_col] = pd.to_datetime(b[date_col], errors="coerce")
    else:
        # Sometimes BoE raw has no explicit date: try index or __source_file mapping
        if isinstance(b.index, pd.DatetimeIndex):
            b = b.reset_index().rename(columns={"index":"date"})
            date_col = "date"
        else:
            date_col = None

    # Try long format (tenor/rate columns exist)
    tenor_col = next((c for c in b.columns if str(c).lower() in {"tenor","maturity","term","pillar"}), None)
    rate_col  = next((c for c in b.columns if str(c).lower() in {"rate","value","ois","spot","par","swap","uk ois spot curve"}), None)

    if date_col and tenor_col and rate_col:
        out = b[[date_col, tenor_col, rate_col]].rename(columns={date_col:"date", tenor_col:"tenor", rate_col:"rate"})
        out["tenor_y"] = out["tenor"].map(_tenor_to_years)
        out["rate"] = pd.to_numeric(out["rate"], errors="coerce")
        out = out.dropna(subset=["date","tenor_y","rate"])
        return out[["date","tenor_y","rate"]].sort_values(["date","tenor_y"]).reset_index(drop=True)

    # Try wide format (date + tenor-like columns)
    if date_col:
        tenor_like = []
        for c in b.columns:
            if c == date_col: 
                continue
            ty = _tenor_to_years(c)
            if np.isfinite(ty):
                tenor_like.append((c, ty))
        if tenor_like:
            cols = [c for c,_ in tenor_like]
            long = b[[date_col] + cols].melt(id_vars=[date_col], var_name="tenor", value_name="rate")
            long["tenor_y"] = long["tenor"].map(_tenor_to_years)
            long["rate"] = pd.to_numeric(long["rate"], errors="coerce")
            out = long.rename(columns={date_col:"date"}).dropna(subset=["date","tenor_y","rate"])
            return out[["date","tenor_y","rate"]].sort_values(["date","tenor_y"]).reset_index(drop=True)

    # Fallback: if the loader already built gbp5 (daily series), store as long at tenor=5
    if "gbp5" in globals() and isinstance(globals()["gbp5"], pd.Series) and isinstance(globals()["gbp5"].index, pd.DatetimeIndex):
        s = globals()["gbp5"].dropna().copy()
        out = pd.DataFrame({"date": s.index, "tenor_y": 5.0, "rate": s.values})
        return out.sort_values(["date","tenor_y"]).reset_index(drop=True)

    raise RuntimeError("Could not canonicalize BoE OIS data into (date, tenor, rate). Inspect boe_df structure.")

# Use already-loaded boe if present; otherwise reload via boe_fp/find_one
if "boe" in globals() and isinstance(boe, pd.DataFrame):
    boe_df = boe
else:
    boe_fp = globals().get("boe_fp", None)
    if boe_fp is None:
        boe_fp = find_one(BASE/'data_clean', ['*boe*ois*.parquet','*ois*daily*raw*.parquet'], 'gbp_curve_raw')
    boe_df = pd.read_parquet(boe_fp)

boe_long = boe_to_long(boe_df)

# Normalize % vs decimal
med = boe_long["rate"].median()
if np.isfinite(med) and med > 1.0:
    boe_long["rate"] = boe_long["rate"] / 100.0

# Save canonical long (for audit)
write_csv_new(boe_long, OUT/'boe_ois_long.csv', index=False)

# Select 5Y explicitly
boe_long["is5"] = np.isclose(boe_long["tenor_y"].astype(float), 5.0, atol=1e-6)
boe_5y = boe_long.loc[boe_long["is5"], ["date","rate"]].drop_duplicates("date").sort_values("date")
if boe_5y.empty:
    raise RuntimeError("No 5Y tenor found in BoE long data. Check outputs/boe_ois_long.csv")

gbp5 = pd.Series(boe_5y["rate"].values, index=pd.to_datetime(boe_5y["date"]).dt.tz_localize(None), name="s5_fund").sort_index()
gbp5 = gbp5[~gbp5.index.duplicated(keep="last")].dropna()

# Save series
write_csv_new(gbp5.reset_index().rename(columns={"index":"date", "s5_fund":"gbp5_fund"}), OUT/'gbp5_funding_series.csv', index=False)

print("GBP5 rebuilt from BoE canonical long:", gbp5.index.min().date(), "→", gbp5.index.max().date(),
      "| missing_frac_daily=", float(gbp5.isna().mean()),
      "| min/median/max=", float(gbp5.min()), float(gbp5.median()), float(gbp5.max()))


GBP5 rebuilt from BoE canonical long: 2009-01-02 → 2026-02-12 | missing_frac_daily= 0.0 | min/median/max= 2.358735926661e-05 0.00482294687585399 0.056929643311827115


In [19]:
# ---- Funding 5Y proxy (GBP): local BoE OIS XLSX files in data_clean/ ----
# Uses only the pasted files:
#   OIS daily data_2009 to 2015.xlsx
#   OIS daily data_2016 to 2024.xlsx
#   OIS daily data_2025 to present.xlsx
# Extracts the 5Y column from the "spot curve" sheet and returns gbp_s5 (daily, decimal).

from pathlib import Path
import numpy as np
import pandas as pd
import re

# Use your existing DATA_CLEAN if defined earlier; otherwise default:
try:
    DATA_CLEAN
except NameError:
    DATA_CLEAN = Path("data_clean")

# Find the daily BoE curve workbooks (handle underscores/spaces)
boe_daily_xlsx = sorted({*DATA_CLEAN.glob("OIS daily data*.xlsx"), *DATA_CLEAN.glob("OIS daily data_*.xlsx")})
if not boe_daily_xlsx:
    raise RuntimeError(f"No 'OIS daily data*.xlsx' files found in {DATA_CLEAN.resolve()}")

def _pick_spot_sheet(xlsx_path: Path) -> str:
    xf = pd.ExcelFile(xlsx_path)
    spot = [s for s in xf.sheet_names if "spot curve" in s.lower()]
    if not spot:
        raise RuntimeError(f"No spot-curve sheet in {xlsx_path.name}. Sheets={xf.sheet_names}")
    # prefer the most detailed spot curve (usually last / '4. spot curve')
    spot = sorted(spot, key=lambda s: (len(s), s))
    return spot[-1]

def _extract_spot_5y_from_file(xlsx_path: Path) -> pd.Series:
    sheet = _pick_spot_sheet(xlsx_path)
    raw = pd.read_excel(xlsx_path, sheet_name=sheet, header=None)

    # locate the "years:" header row
    c0 = raw.iloc[:, 0].astype(str).str.strip().str.lower()
    years_rows = c0[c0.str.match(r"^years\b")].index.to_list()
    if not years_rows:
        raise RuntimeError(f"Could not find 'years:' header row in {xlsx_path.name}::{sheet}")
    yrow = years_rows[0]

    # locate the column whose years value is ~5.0
    hdr = raw.iloc[yrow, :]
    col5 = None
    for j, v in hdr.items():
        if isinstance(v, (int, float, np.floating)) and np.isfinite(v) and abs(float(v) - 5.0) < 1e-2:
            col5 = j
            break
    if col5 is None:
        # common in 2009–2015 file: 4.999999999999999
        for j, v in hdr.items():
            if isinstance(v, (int, float, np.floating)) and np.isfinite(v) and abs(float(v) - 5.0) < 5e-2:
                col5 = j
                break
    if col5 is None:
        raise RuntimeError(f"No ~5Y pillar found in {xlsx_path.name}::{sheet}")

    # data begins below the header rows; first col is date
    dates = pd.to_datetime(raw.iloc[yrow + 1 :, 0], errors="coerce")
    rates = pd.to_numeric(raw.iloc[yrow + 1 :, col5], errors="coerce")

    s = pd.Series(rates.to_numpy(), index=dates, name="s5_fund").dropna()
    s.index = pd.to_datetime(s.index, errors="coerce").tz_localize(None)
    s = s[~s.index.duplicated(keep="last")].sort_index()
    return s

parts, failed = [], []
for fp in boe_daily_xlsx:
    try:
        parts.append(_extract_spot_5y_from_file(fp))
    except Exception as e:
        failed.append((fp.name, str(e)))

if not parts:
    msg = "Failed to extract GBP 5Y from all BoE daily XLSX files.\n" + "\n".join([f"- {n}: {err}" for n, err in failed])
    raise RuntimeError(msg)

gbp_s5 = pd.concat(parts).sort_index()
gbp_s5 = gbp_s5[~gbp_s5.index.duplicated(keep="last")]

# Convert percent -> decimal (BoE sheets are in percent points, e.g. 0.48 = 0.48%)
if gbp_s5.median() > 1.0 or gbp_s5.quantile(0.95) > 1.0:
    gbp_s5 = gbp_s5 / 100.0

gbp_s5.name = "s5_fund"

print(
    "GBP 5Y (BoE OIS spot curve):",
    gbp_s5.index.min().date(), "to", gbp_s5.index.max().date(),
    "| n=", len(gbp_s5),
    "| median=", float(gbp_s5.median()),
)


GBP 5Y (BoE OIS spot curve): 2009-01-02 to 2026-01-30 | n= 4314 | median= 0.01212514346218281


WEEKLY ALIGNMENT (W-WED) + CURVE DENSIFICATION
- FX/OIS aligned to Wednesday within ±2 days (holiday tolerance).
- Curves densified to business days via time interpolation (audit), then sampled weekly.
Produces:
  weekly_targets
  ois_w, gbp5_w : pd.Series weekly
  fx_w          : pd.DataFrame weekly (USD per CCY)
  curve_w       : dict[ccy] -> pd.DataFrame weekly, columns=tenor_y
  align_audit, staleness_summary, interp_audit

In [ ]:
MAX_NEAR_DAYS = 2                 # FX/OIS holiday tolerance
TIME_INTERP_MAX_GAP_DAYS = 45     # max consecutive business-day NaNs to interpolate
WEEKLY_CURVE_MAX_STALE_DAYS = 7   # max staleness for curve sampling after densification

def _nearest_within(idx: pd.DatetimeIndex, d: pd.Timestamp, max_days: int):
    if len(idx) == 0:
        return None
    pos = idx.get_indexer([d], method="nearest")[0]
    if pos < 0:
        return None
    dd = idx[pos]
    return dd if abs((dd - d).days) <= max_days else None

def align_series_weekly_nearest(s: pd.Series, targets: pd.DatetimeIndex, max_days: int):
    s = s.dropna().sort_index()
    idx = s.index
    out, used = [], []
    for d in targets:
        dd = _nearest_within(idx, d, max_days)
        out.append(np.nan if dd is None else float(s.loc[dd]))
        used.append(pd.NaT if dd is None else dd)
    return pd.Series(out, index=targets, name=s.name), pd.Series(used, index=targets, name=f"{s.name}_used_date")

def densify_curves_to_bdays(curves_long: pd.DataFrame):
    curves = curves_long.copy()
    curves["date"] = pd.to_datetime(curves["date"]).dt.tz_localize(None)
    curves = curves.sort_values(["ccy","tenor_y","date"])

    d0, d1 = curves["date"].min(), curves["date"].max()
    cal = pd.date_range(d0, d1, freq="B")

    audit, parts = [], []
    for (ccy, ten), g in curves.groupby(["ccy","tenor_y"]):
        s = pd.Series(g["par_swap"].to_numpy(), index=g["date"]).sort_index()
        s = s[~s.index.duplicated(keep="last")]
        s2 = s.reindex(cal)
        s3 = s2.interpolate(method="time", limit=TIME_INTERP_MAX_GAP_DAYS, limit_area="inside")

        exact = s3.index.isin(s.index) & s3.notna()
        audit.append({
            "ccy": ccy,
            "tenor_y": float(ten),
            "n_obs_raw": int(s.notna().sum()),
            "n_bdays": int(s3.notna().sum()),
            "interp_share": float((s3.notna().sum() - exact.sum()) / max(int(s3.notna().sum()), 1)),
            "max_gap_days_raw": int(pd.Series(s.index).diff().dt.days.max()) if len(s.index) > 1 else 0,
        })

        parts.append(pd.DataFrame({"date": s3.index, "ccy": ccy, "tenor_y": float(ten), "par_swap": s3.values}))

    dense = pd.concat(parts, ignore_index=True).dropna(subset=["par_swap"])
    curve_panel_b = dense.pivot_table(index=["date","ccy"], columns="tenor_y", values="par_swap", aggfunc="mean").sort_index()
    return curve_panel_b, pd.DataFrame(audit)

curve_panel_b, interp_audit = densify_curves_to_bdays(curves_long)

curve_min = curve_panel_b.index.get_level_values("date").min()
curve_max = curve_panel_b.index.get_level_values("date").max()
start = max(ois_on.index.min(), gbp_s5.index.min(), fx_usd_per_ccy.index.min(), curve_min)
end   = min(ois_on.index.max(), gbp_s5.index.max(), fx_usd_per_ccy.index.max(), curve_max)

weekly_targets = pd.date_range(start, end, freq="W-WED")
if len(weekly_targets) < 150:
    raise RuntimeError(f"Weekly sample too short: n={len(weekly_targets)}")

ois_w, ois_used = align_series_weekly_nearest(ois_on, weekly_targets, MAX_NEAR_DAYS)
gbp5_w, gbp5_used = align_series_weekly_nearest(gbp_s5, weekly_targets, MAX_NEAR_DAYS)

fx_w = pd.DataFrame(index=weekly_targets)
fx_used = {}
for c in sorted(set(EM + [FUND])):
    fx_w[c], fx_used[c] = align_series_weekly_nearest(fx_usd_per_ccy[c], weekly_targets, MAX_NEAR_DAYS)

def align_curve_weekly(ccy: str, targets: pd.DatetimeIndex, max_stale_days: int):
    panel = curve_panel_b.xs(ccy, level="ccy", drop_level=False)
    idx = panel.index.get_level_values("date").unique().sort_values()
    rows, used = [], []
    for d in targets:
        dd = _nearest_within(idx, d, max_stale_days)
        if dd is None:
            rows.append(pd.Series(np.nan, index=curve_panel_b.columns))
            used.append(pd.NaT)
        else:
            rows.append(panel.xs(dd, level="date").iloc[0])
            used.append(dd)
    out = pd.DataFrame(rows, index=targets, columns=curve_panel_b.columns)
    return out, pd.Series(used, index=targets, name=f"{ccy}_curve_used_date")

curve_w, curve_used = {}, {}
for c in EM:
    curve_w[c], curve_used[c] = align_curve_weekly(c, weekly_targets, WEEKLY_CURVE_MAX_STALE_DAYS)

align_audit = pd.DataFrame({"date": weekly_targets})
align_audit["ois_used_date"] = ois_used.values
align_audit["gbp5_used_date"] = gbp5_used.values
for c in sorted(set(EM + [FUND])):
    align_audit[f"fx_{c}_used_date"] = fx_used[c].values
for c in EM:
    align_audit[f"curve_{c}_used_date"] = curve_used[c].values

def _lag_days(used: pd.Series, tgt: pd.Series) -> pd.Series:
    return (pd.to_datetime(tgt) - pd.to_datetime(used)).dt.days

stale_rows = []
for ucol in [c for c in align_audit.columns if c.endswith("_used_date")]:
    lag = _lag_days(align_audit[ucol], align_audit["date"])
    stale_rows.append({
        "series": ucol.replace("_used_date",""),
        "n_weeks": int(lag.notna().sum()),
        "mean_lag_days": float(lag.mean()),
        "p95_lag_days": float(lag.quantile(0.95)) if lag.notna().any() else np.nan,
        "max_lag_days": float(lag.max()) if lag.notna().any() else np.nan,
    })
staleness_summary = pd.DataFrame(stale_rows).sort_values("series").reset_index(drop=True)

print("Weekly panel:",
      "| weeks", len(weekly_targets),
      "| start", weekly_targets.min().date(),
      "| end", weekly_targets.max().date())


Weekly panel: | weeks 366 | start 2019-01-02 | end 2025-12-31


PRICING + SIMULATION (faithful to spec + guardrails + decomposition)
- quarterly coupon bond, coupon fixed at entry s5_lend
- convert all flows to USD; compute equity return and notional return
- enforce "use at least 1Y and 5Y points": require tenor range covers [1,5] at entry AND exit
- audit: bootstrap conventions + entry par check
Produces:
  trades : per-ccy-week panel
  wk     : weekly portfolio panel (variable-capital + fixed-allocation)

In [ ]:
NOTIONAL_USD = 10_000_000.0
LEVERAGE = 5.0
BORROW_USD = NOTIONAL_USD * (LEVERAGE - 1.0) / LEVERAGE   # 8mm
EQUITY_USD = NOTIONAL_USD / LEVERAGE                       # 2mm
DT = 1.0 / 52.0
COUPON_FREQ = 4
SPREAD_BPS = 50.0
SPREAD = SPREAD_BPS / 10_000.0

times_entry = np.arange(1/COUPON_FREQ, 5.0 + 1e-12, 1/COUPON_FREQ)
times_exit = times_entry - DT
times_exit = times_exit[times_exit > 0]

def _interp_linear(x: np.ndarray, y: np.ndarray, t: float) -> float:
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size == 0:
        return np.nan
    o = np.argsort(x); x, y = x[o], y[o]
    return float(np.interp(t, x, y, left=np.nan, right=np.nan))

def _require_1y_5y(curve: pd.Series) -> bool:
    if curve is None or curve.dropna().size < 2:
        return False
    ten = curve.dropna().index.astype(float).to_numpy()
    lo, hi = float(np.nanmin(ten)), float(np.nanmax(ten))
    return (lo <= 1.0) and (hi >= 5.0)

def _interp_par_with_short_end(tenors: np.ndarray, rates: np.ndarray, t: float):
    x = np.asarray(tenors, float); y = np.asarray(rates, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, {"short_used": 0, "long_forbidden": 1}
    o = np.argsort(x); x, y = x[o], y[o]
    lo, hi = float(x[0]), float(x[-1])
    if t < lo:
        return float(y[0]), {"short_used": 1, "long_forbidden": 0}
    if t > hi:
        return np.nan, {"short_used": 0, "long_forbidden": 1}
    return float(np.interp(t, x, y)), {"short_used": 0, "long_forbidden": 0}

def bootstrap_df(par_curve: pd.Series, max_t: float, freq: int = 4):
    grid = np.arange(1/freq, max_t + 1e-12, 1/freq)
    par_curve = par_curve.dropna()
    ten = par_curve.index.astype(float).to_numpy()
    rat = par_curve.astype(float).to_numpy()
    m = np.isfinite(ten) & np.isfinite(rat)
    ten, rat = ten[m], rat[m]
    if ten.size < 2:
        return pd.Series(dtype=float), {"short_used_ct": 0, "long_forbidden_ct": len(grid), "min_tenor": np.nan, "max_tenor": np.nan}
    o = np.argsort(ten); ten, rat = ten[o], rat[o]
    audit = {"short_used_ct": 0, "long_forbidden_ct": 0, "min_tenor": float(ten[0]), "max_tenor": float(ten[-1])}

    dfs = {}
    for t in grid:
        s, a = _interp_par_with_short_end(ten, rat, float(t))
        audit["short_used_ct"] += a["short_used"]
        audit["long_forbidden_ct"] += a["long_forbidden"]
        if not np.isfinite(s):
            return pd.Series(dtype=float), audit
        c = s / freq
        prev = sum((c * dfs[pt]) for pt in grid if pt < t)
        dfs[t] = max((1 - prev) / (1 + c), 1e-12)

    return pd.Series(dfs), audit

def price_bond(coupon: float, curve_row: pd.Series, times: np.ndarray, freq: int = 4):
    if times.size == 0:
        return np.nan, {"short_used_ct": 0, "long_forbidden_ct": 1, "min_tenor": np.nan, "max_tenor": np.nan}
    max_t = max(5.0, float(np.max(times)) + 0.25)
    z, aud = bootstrap_df(curve_row, max_t=max_t, freq=freq)
    if z.empty:
        return np.nan, aud
    df = np.interp(times, z.index.values, z.values, left=z.values[0], right=z.values[-1])
    if np.isnan(df).any():
        return np.nan, aud
    cf = np.full(times.size, coupon/freq)
    cf[-1] += 1.0
    return float(np.sum(cf * df)), aud

rows, wk_rows, par_check, audit_rows = [], [], [], []

for i in range(len(weekly_targets) - 1):
    t0, t1 = weekly_targets[i], weekly_targets[i+1]

    s5_fund = float(gbp5_w.loc[t0]) if np.isfinite(gbp5_w.loc[t0]) else np.nan
    ois0 = float(ois_w.loc[t0]) if np.isfinite(ois_w.loc[t0]) else np.nan

    active_rets_equity, active_rets_notional, active_pnls = [], [], []
    n_avail, n_active = 0, 0

    for c in EM:
        fx0 = float(fx_w.at[t0, c]) if np.isfinite(fx_w.at[t0, c]) else np.nan
        fx1 = float(fx_w.at[t1, c]) if np.isfinite(fx_w.at[t1, c]) else np.nan
        g0  = float(fx_w.at[t0, FUND]) if np.isfinite(fx_w.at[t0, FUND]) else np.nan
        g1  = float(fx_w.at[t1, FUND]) if np.isfinite(fx_w.at[t1, FUND]) else np.nan

        c0 = curve_w[c].loc[t0].dropna()
        c1 = curve_w[c].loc[t1].dropna()

        available, reason = True, None
        if not np.isfinite([fx0, fx1, g0, g1, s5_fund, ois0]).all():
            available, reason = False, "missing_fx_or_funding"
        if available and (c0.empty or c1.empty):
            available, reason = False, "missing_curve"
        if available and (not _require_1y_5y(c0) or not _require_1y_5y(c1)):
            available, reason = False, "no_1y_5y_coverage"

        s5_lend = np.nan
        spread = np.nan
        active = False
        ret_equity = np.nan
        ret_notional = np.nan
        pnl_usd = 0.0

        lend_pnl_usd = np.nan
        lend_pnl_rates = np.nan
        lend_pnl_fx = np.nan
        debt_cost_usd = np.nan
        debt_cost_interest = np.nan
        debt_cost_fx = np.nan

        if available:
            n_avail += 1
            ten0 = c0.index.astype(float).to_numpy()
            rat0 = c0.astype(float).to_numpy()
            s5_lend = _interp_linear(ten0, rat0, 5.0)
            if not np.isfinite(s5_lend):
                available, reason = False, "s5_nan"
            else:
                spread = s5_lend - s5_fund
                active = bool(np.isfinite(spread) and spread >= SPREAD)

        if available and active:
            pv0, aud0 = price_bond(s5_lend, c0, times_entry, freq=COUPON_FREQ)
            par_check.append({"date": t0, "ccy": c, "pv_entry": pv0, "abs_err": abs(pv0 - 1.0) if np.isfinite(pv0) else np.nan,
                              "min_tenor": aud0.get("min_tenor"), "max_tenor": aud0.get("max_tenor")})
            if not (np.isfinite(pv0) and abs(pv0 - 1.0) <= 5e-3):
                available, active, reason = False, False, "entry_par_check_failed"

        if available and active:
            pv1, aud1 = price_bond(s5_lend, c1, times_exit, freq=COUPON_FREQ)
            audit_rows.append({"date": t0, "ccy": c, **aud1})
            if not np.isfinite(pv1):
                available, active, reason = False, False, "pricing_nan"

        if available and active:
            face_ccy = NOTIONAL_USD / fx0
            lend_usd_exit = face_ccy * pv1 * fx1
            lend_pnl_usd = lend_usd_exit - NOTIONAL_USD

            lend_usd_exit_fx0 = face_ccy * pv1 * fx0
            lend_pnl_rates = lend_usd_exit_fx0 - NOTIONAL_USD
            lend_pnl_fx = lend_usd_exit - lend_usd_exit_fx0

            debt_gbp = BORROW_USD / g0
            repay_gbp = debt_gbp * (1.0 + (ois0 + SPREAD) * DT)
            repay_usd = repay_gbp * g1

            debt_cost_usd = repay_usd - BORROW_USD
            repay_usd_fx0 = repay_gbp * g0
            debt_cost_interest = repay_usd_fx0 - BORROW_USD
            debt_cost_fx = repay_usd - repay_usd_fx0

            eq_exit = lend_usd_exit - repay_usd
            pnl_usd = eq_exit - EQUITY_USD
            ret_equity = max(pnl_usd / EQUITY_USD, -0.999999)
            ret_notional = pnl_usd / NOTIONAL_USD

            n_active += 1
            active_rets_equity.append(ret_equity)
            active_rets_notional.append(ret_notional)
            active_pnls.append(pnl_usd)

        rows.append({
            "date": t0, "next_date": t1, "ccy": c,
            "available": int(available), "active": int(active),
            "reason_unavailable": reason,
            "s5_lend": s5_lend, "s5_fund": s5_fund, "spread": spread,
            "ret_equity": ret_equity, "ret_notional": ret_notional,
            "pnl_usd": pnl_usd,
            "lend_pnl_usd": lend_pnl_usd, "lend_pnl_rates": lend_pnl_rates, "lend_pnl_fx": lend_pnl_fx,
            "debt_cost_usd": debt_cost_usd, "debt_cost_interest": debt_cost_interest, "debt_cost_fx": debt_cost_fx,
        })

    wk_rows.append({
        "date": t0, "next_date": t1,
        "n_ccy_available": int(n_avail), "n_ccy_active": int(n_active),
        "port_ret_equity": float(np.mean(active_rets_equity)) if active_rets_equity else 0.0,
        "port_ret_notional": float(np.mean(active_rets_notional)) if active_rets_notional else 0.0,
        "port_pnl_usd": float(np.sum(active_pnls)) if active_pnls else 0.0,
        "port_equity_usd": float(EQUITY_USD * n_active),
    })

trades = pd.DataFrame(rows).sort_values(["date","ccy"]).reset_index(drop=True)
wk = pd.DataFrame(wk_rows).set_index("date").sort_index()

# --- Ensure wk has portfolio return series (compute from trades if missing) ---
import numpy as np
import pandas as pd

# wk must be indexed by weekly dates; if it's not, make it so.
if "date" in wk.columns and not isinstance(wk.index, pd.DatetimeIndex):
    wk = wk.set_index("date")
wk.index = pd.to_datetime(wk.index, errors="coerce").tz_localize(None)
wk = wk.sort_index()

# Wide matrix of per-ccy equity returns (NaN when not active)
r_wide = trades.pivot(index="date", columns="ccy", values="ret_equity")
r_wide.index = pd.to_datetime(r_wide.index, errors="coerce").tz_localize(None)
r_wide = r_wide.reindex(wk.index)

# Active indicator (1/0)
a_wide = trades.pivot(index="date", columns="ccy", values="active")
a_wide.index = pd.to_datetime(a_wide.index, errors="coerce").tz_localize(None)
a_wide = a_wide.reindex(wk.index).fillna(0.0)

# Variable-capital: average across ACTIVE positions each week (0 if none active)
if "port_ret_equity" not in wk.columns:
    denom = a_wide.sum(axis=1).replace(0.0, np.nan)
    wk["port_ret_equity"] = (r_wide.fillna(0.0) * a_wide).sum(axis=1) / denom
    wk["port_ret_equity"] = wk["port_ret_equity"].fillna(0.0)

# Fixed-allocation: equal weight across currencies each week, inactive treated as 0
if "port_ret_equity_fixed" not in wk.columns:
    wk["port_ret_equity_fixed"] = r_wide.fillna(0.0).mean(axis=1)

# Wealth/drawdown convenience (used by figures/report)
if "wealth_equity" not in wk.columns:
    wk["wealth_equity"] = (1.0 + wk["port_ret_equity"].fillna(0.0)).cumprod()
if "drawdown_equity" not in wk.columns:
    w_ = wk["wealth_equity"]
    wk["drawdown_equity"] = w_ / w_.cummax() - 1.0

if "wealth_equity_fixed" not in wk.columns:
    wk["wealth_equity_fixed"] = (1.0 + wk["port_ret_equity_fixed"].fillna(0.0)).cumprod()
if "drawdown_equity_fixed" not in wk.columns:
    w_ = wk["wealth_equity_fixed"]
    wk["drawdown_equity_fixed"] = w_ / w_.cummax() - 1.0


wk["wealth_equity"] = (1.0 + wk["port_ret_equity"].fillna(0.0)).cumprod()
wk["drawdown_equity"] = wk["wealth_equity"] / wk["wealth_equity"].cummax() - 1.0

wide_eq = trades.pivot(index="date", columns="ccy", values="ret_equity").reindex(wk.index).fillna(0.0)
wk["port_ret_equity_fixed"] = wide_eq.mean(axis=1)
wk["wealth_equity_fixed"] = (1.0 + wk["port_ret_equity_fixed"]).cumprod()
wk["drawdown_equity_fixed"] = wk["wealth_equity_fixed"] / wk["wealth_equity_fixed"].cummax() - 1.0

print("Simulation:",
      "| weeks", len(wk),
      "| avg_active", float(wk["n_ccy_active"].mean()),
      "| min_dd_equity_fixed", float(wk["drawdown_equity_fixed"].min()))


Simulation: | weeks 365 | avg_active 1.7671232876712328 | min_dd_equity_fixed -0.8420760158312669


In [28]:
# --- Ensure weekly GBP 5Y series exists under name gbp5_w (used later in factor block) ---
if "gbp5_w" not in locals():
    if "gbp_s5_w" in locals():
        gbp5_w = gbp_s5_w.rename("s5_fund")
    else:
        # if you only have daily gbp_s5, create weekly
        if "gbp_s5" in locals():
            gbp5_w = gbp_s5.resample("W-WED").last().reindex(wk.index).rename("s5_fund")
        else:
            raise RuntimeError("Missing GBP 5Y series: expected gbp_s5_w or gbp_s5")


In [ ]:
def s5_strict(curve_series: pd.Series) -> float:
    """Return 5Y par rate only if 5.0 is within tenor range; otherwise NaN."""
    if curve_series is None or curve_series.dropna().empty:
        return np.nan
    x = curve_series.dropna().index.astype(float).to_numpy()
    y = curve_series.dropna().astype(float).to_numpy()
    o = np.argsort(x); x, y = x[o], y[o]
    if (5.0 < x[0]) or (5.0 > x[-1]):
        return np.nan
    return float(np.interp(5.0, x, y))

def interp_par_with_short_end(tenors: np.ndarray, rates: np.ndarray, t: float) -> tuple[float, dict]:
    """
    Par-rate interpolation with explicit conventions:
      - t < min_tenor: flat at rate(min_tenor)  (short-end convention)
      - t > max_tenor: NaN (forbid extrapolation)
    Returns (rate, audit)
    """
    x = np.asarray(tenors, float)
    y = np.asarray(rates, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, {"short_used": 0, "long_forbidden": 1}
    o = np.argsort(x); x, y = x[o], y[o]
    lo, hi = float(x[0]), float(x[-1])
    if t < lo:
        return float(y[0]), {"short_used": 1, "long_forbidden": 0}
    if t > hi:
        return np.nan, {"short_used": 0, "long_forbidden": 1}
    return float(np.interp(t, x, y)), {"short_used": 0, "long_forbidden": 0}

def bootstrap_df_explicit(par_curve: pd.Series, freq: int = 4, max_t: float = 5.0) -> tuple[pd.Series, dict]:
    """
    Bootstrap discount factors on a quarterly grid up to max_t using par rates.
    Uses explicit short-end convention and forbids long-end extrapolation.
    Returns (df_series, audit_dict).
    """
    grid = np.arange(1/freq, max_t + 1e-12, 1/freq)
    dfs = {}
    audit = {"short_used_ct": 0, "long_forbidden_ct": 0, "min_tenor": np.nan, "max_tenor": np.nan}

    ten = par_curve.index.astype(float).to_numpy()
    rat = par_curve.astype(float).to_numpy()
    m = np.isfinite(ten) & np.isfinite(rat)
    ten, rat = ten[m], rat[m]
    if ten.size < 2:
        return pd.Series(dtype=float), audit
    o = np.argsort(ten); ten, rat = ten[o], rat[o]
    audit["min_tenor"], audit["max_tenor"] = float(ten[0]), float(ten[-1])

    for t in grid:
        s, a = interp_par_with_short_end(ten, rat, float(t))
        audit["short_used_ct"] += a["short_used"]
        audit["long_forbidden_ct"] += a["long_forbidden"]
        if not np.isfinite(s):
            return pd.Series(dtype=float), audit
        c = s / freq
        prev = sum((c * dfs[pt]) for pt in grid if pt < t)
        dfs[t] = max((1 - prev) / (1 + c), 1e-12)

    return pd.Series(dfs), audit

def price_bond_mtm(coupon: float, curve_row: pd.Series, times: np.ndarray, freq: int = 4) -> tuple[float, dict]:
    """
    Price a quarterly coupon bond with given remaining cashflow times.
    Uses explicit bootstrap; returns (pv, audit).
    """
    if times.size == 0:
        return np.nan, {"short_used_ct": 0, "long_forbidden_ct": 1}
    max_t = max(5.0, float(np.max(times)) + 0.25)
    z, audit = bootstrap_df_explicit(curve_row.dropna(), freq=freq, max_t=max_t)
    if z.empty:
        return np.nan, audit
    # DF interpolation: still forbid beyond max curve by z construction
    df = np.interp(times, z.index.values, z.values, left=z.values[0], right=z.values[-1])
    if np.isnan(df).any():
        return np.nan, audit
    cf = np.full(times.size, coupon/freq)
    cf[-1] += 1.0
    return float(np.sum(cf * df)), audit

# Re-run strategy simulation using aligned weekly objects built earlier
times_entry = np.arange(0.25, 5.0 + 1e-12, 0.25)
dt = 1/52
times_exit = (times_entry - dt)
times_exit = times_exit[times_exit > 0]

rows = []
weekly = []
audit_rows = []

missing_driver = []

for i in range(len(weekly_targets) - 1):
    t0 = weekly_targets[i]
    t1 = weekly_targets[i+1]

    s5f = gbp5_w.loc[t0]
    o = ois_w.loc[t0]

    active = 0
    avail = 0
    rets = []

    for c in EM:
        fx0, fx1 = fx_w.at[t0, c], fx_w.at[t1, c]
        g0, g1   = fx_w.at[t0, "GBP"], fx_w.at[t1, "GBP"]
        c0 = curve_w[c].loc[t0].dropna()
        c1 = curve_w[c].loc[t1].dropna()

        ok = True
        if not np.isfinite([fx0, fx1, g0, g1]).all():
            ok = False; missing_driver.append((t0, c, "fx_missing"))
        if c0.empty or c1.empty:
            ok = False; missing_driver.append((t0, c, "curve_missing"))
        if not np.isfinite(s5f):
            ok = False; missing_driver.append((t0, c, "gbp5_missing"))
        if not np.isfinite(o):
            ok = False; missing_driver.append((t0, c, "ois_missing"))

        ret = np.nan
        pnl = 0.0
        trade = False
        s5l = np.nan
        spread = np.nan

        if ok:
            # STRICT 5Y availability at entry
            s5l = s5_strict(c0)
            if not np.isfinite(s5l):
                ok = False
                missing_driver.append((t0, c, "no_5y_in_range"))
            else:
                avail += 1
                spread = s5l - s5f
                trade = bool(np.isfinite(spread) and spread >= 0.005)

                if trade:
                    pv1, aud = price_bond_mtm(s5l, c1, times_exit)
                    audit_rows.append({"date": t0, "ccy": c, **aud})

                    if np.isfinite(pv1):
                        lend_end = (10_000_000 / fx0) * pv1 * fx1
                        debt_end = (8_000_000 / g0) * (1 + (o + 0.005)/52) * g1
                        eq1 = lend_end - debt_end
                        ret = max((eq1 - 2_000_000) / 2_000_000, -0.999999)
                        pnl = (eq1 - 2_000_000)
                        active += 1
                        rets.append(ret)
                    else:
                        trade = False
                        missing_driver.append((t0, c, "pricing_nan"))

        rows.append({
            "date": t0, "next_date": t1, "ccy": c,
            "available": int(ok), "active": int(trade),
            "s5_lend": s5l, "s5_fund": s5f, "spread": spread,
            "ret": ret, "pnl_usd": pnl
        })

    weekly.append({
        "date0": t0, "date1": t1,
        "n_ccy_available": avail, "n_ccy_active": active,
        "port_ret": float(np.mean(rets)) if rets else 0.0,
        "active_positions": active
    })

res = pd.DataFrame(rows)
wk = pd.DataFrame(weekly).set_index("date0").sort_index()
wk["wealth"] = (1 + wk["port_ret"]).cumprod()
wk["drawdown"] = wk["wealth"] / wk["wealth"].cummax() - 1

# Save extrapolation audit summary
if audit_rows:
    aud = pd.DataFrame(audit_rows)
    write_csv_new(aud, OUT/"extrapolation_audit_raw.csv", index=False)
    summ = aud.groupby("ccy").agg(
        n=("date","count"),
        short_used=("short_used_ct","mean"),
        long_forbidden=("long_forbidden_ct","mean"),
    ).reset_index()
    write_csv_new(summ, OUT/"extrapolation_audit_summary.csv", index=False)
else:
    write_csv_new(pd.DataFrame([], columns=["date","ccy","short_used_ct","long_forbidden_ct"]), OUT/"extrapolation_audit_raw.csv", index=False)
    write_csv_new(pd.DataFrame([], columns=["ccy","n","short_used","long_forbidden"]), OUT/"extrapolation_audit_summary.csv", index=False)

# Diagnostics driver dump if needed
md = pd.DataFrame(missing_driver, columns=["date","ccy","driver"])
write_csv_new(md, OUT/"missing_drivers.csv", index=False)

print("NEW simulation complete:",
      "| weeks=", len(wk),
      "| avg_active_positions=", float(wk["active_positions"].mean()),
      "| min_drawdown=", float(wk["drawdown"].min()))


Backed up existing outputs\extrapolation_audit_raw.csv -> outputs\extrapolation_audit_raw.csv.bak_20260217_234018
Backed up existing outputs\extrapolation_audit_summary.csv -> outputs\extrapolation_audit_summary.csv.bak_20260217_234018
Backed up existing outputs\missing_drivers.csv -> outputs\missing_drivers.csv.bak_20260217_234018
NEW simulation complete: | weeks= 365 | avg_active_positions= 2.8109589041095893 | min_drawdown= -0.9953604737695458


In [ ]:
BASE = Path(".")
OUT = BASE / "outputs"
FIG = OUT / "figures"
OUT.mkdir(parents=True, exist_ok=True)
FIG.mkdir(parents=True, exist_ok=True)

def _write_csv(df: pd.DataFrame, fp: Path, index: bool = False) -> None:
    fp.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(fp, index=index)

def _write_text(text: str, fp: Path) -> None:
    fp.write_text(text, encoding="utf-8")

def _sharpe_w(r: pd.Series) -> float:
    r = r.dropna().astype(float)
    s = r.std(ddof=0)
    return float(r.mean() / s) if np.isfinite(s) and s > 0 else np.nan

def _wealth_dd(r: pd.Series):
    w = (1.0 + r.fillna(0.0).astype(float)).cumprod()
    dd = w / w.cummax() - 1.0
    return w, dd

def _md_table(df: pd.DataFrame, floatfmt: str = "{:.6f}", index: bool = False) -> str:
    d = df.copy()
    # ensure simple scalar formatting
    for c in d.columns:
        if pd.api.types.is_float_dtype(d[c]):
            d[c] = d[c].map(lambda x: "" if pd.isna(x) else floatfmt.format(float(x)))
    # manual markdown (no tabulate)
    cols = (["index"] + list(d.columns)) if index else list(d.columns)
    header = "| " + " | ".join(cols) + " |"
    sep = "|" + "|".join(["---"] * len(cols)) + "|"
    lines = [header, sep]
    if index:
        for ix, row in d.iterrows():
            vals = [str(ix)] + [("" if pd.isna(v) else str(v)) for v in row.values.tolist()]
            lines.append("| " + " | ".join(vals) + " |")
    else:
        for _, row in d.iterrows():
            vals = [("" if pd.isna(v) else str(v)) for v in row.values.tolist()]
            lines.append("| " + " | ".join(vals) + " |")
    return "\n".join(lines)

# ----------------------------
# Guardrails: required objects
# ----------------------------
for name in ["wk", "res", "fx_w", "ois_w", "gbp5_w", "EM"]:
    if name not in globals():
        raise RuntimeError(f"Missing required object `{name}` in memory. Ensure NEW CELL B ran successfully.")

wk2 = wk.copy().sort_index()
# wk2 index should be t0 weeks (length N-1)
wk2.index = pd.to_datetime(wk2.index)
wk2.index.name = "date0"

# ----------------------------
# 1) Portfolio outputs
# ----------------------------
port_out = wk2.reset_index().rename(columns={"date0": "date"})
port_out["date"] = pd.to_datetime(port_out["date"])
port_out["wealth"] = (1.0 + port_out["port_ret"].fillna(0.0)).cumprod()
port_out["drawdown"] = port_out["wealth"] / port_out["wealth"].cummax() - 1.0

cal_out = port_out[["date", "date1", "n_ccy_available", "n_ccy_active"]].copy().rename(columns={"date": "date0"})
_write_csv(cal_out, OUT / "weekly_calendar.csv", index=False)

_write_csv(
    port_out[["date", "port_ret", "wealth", "drawdown", "active_positions"]],
    OUT / "portfolio_weekly_returns.csv",
    index=False
)

# ----------------------------
# 2) Currency stats
# ----------------------------
tab = []
for c in EM:
    rc = res.loc[res["ccy"] == c].copy()
    wa = int(len(rc))
    wt = int((rc["active"].astype(int) == 1).sum())
    af = (wt / wa) if wa else np.nan

    rca = rc.loc[rc["active"].astype(int) == 1, "ret"].dropna().astype(float)
    ru = rc["ret"].astype(float).fillna(0.0)

    sh = _sharpe_w(rca) if len(rca) > 1 else np.nan
    _, dd_c = _wealth_dd(ru)

    tab.append({
        "ccy": c,
        "weeks_available": wa,
        "weeks_traded": wt,
        "active_frac": af,
        "mean_weekly_ret_cond_active": float(rca.mean()) if len(rca) else np.nan,
        "vol_weekly_ret_cond_active": float(rca.std(ddof=0)) if len(rca) > 1 else np.nan,
        "mean_weekly_ret_uncond": float(ru.mean()),
        "vol_weekly_ret_uncond": float(ru.std(ddof=0)),
        "sharpe_weekly_cond_active": float(sh) if np.isfinite(sh) else np.nan,
        "sharpe_ann_cond_active": float(np.sqrt(52) * sh) if np.isfinite(sh) else np.nan,
        "pnl_sum_usd": float(rc["pnl_usd"].fillna(0.0).sum()),
        "max_dd_wealth": float(dd_c.min()),
    })

currency_stats = pd.DataFrame(tab)
_write_csv(currency_stats, OUT / "currency_stats.csv", index=False)

# ----------------------------
# 3) Active diagnostics (clean PORTFOLIO row; spreads NaN)
# ----------------------------
panel = res[["date", "ccy", "available", "active", "spread"]].copy()
panel["date"] = pd.to_datetime(panel["date"])
panel["available"] = panel["available"].astype(int).astype(bool)
panel["active"] = panel["active"].astype(int).astype(bool)
panel["spread"] = pd.to_numeric(panel["spread"], errors="coerce")

ad = panel.groupby("ccy").agg(
    weeks_available=("available", "sum"),
    weeks_traded=("active", "sum"),
    spread_mean=("spread", "mean"),
    spread_p05=("spread", lambda s: float(np.nanquantile(s.dropna(), 0.05)) if s.dropna().size else np.nan),
    spread_p50=("spread", lambda s: float(np.nanquantile(s.dropna(), 0.50)) if s.dropna().size else np.nan),
    spread_p95=("spread", lambda s: float(np.nanquantile(s.dropna(), 0.95)) if s.dropna().size else np.nan),
).reset_index().rename(columns={"ccy": "entity"})
ad["active_frac"] = ad["weeks_traded"] / ad["weeks_available"].replace(0, np.nan)

weeks_available = int(port_out["date"].nunique())
weeks_with_any_position = int((port_out["active_positions"] > 0).sum())
avg_active_positions = float(port_out["active_positions"].mean())

port_row = pd.DataFrame([{
    "entity": "PORTFOLIO",
    "weeks_available": weeks_available,
    "weeks_traded": weeks_with_any_position,
    "active_frac": weeks_with_any_position / weeks_available if weeks_available else np.nan,
    "weeks_with_any_position": weeks_with_any_position,
    "avg_active_positions": avg_active_positions,
    "spread_mean": np.nan,
    "spread_p05": np.nan,
    "spread_p50": np.nan,
    "spread_p95": np.nan,
}])

ad["weeks_with_any_position"] = np.nan
ad["avg_active_positions"] = np.nan
active_diag = pd.concat([ad, port_row], ignore_index=True)
_write_csv(active_diag, OUT / "active_diagnostics.csv", index=False)

# ----------------------------
# 4) Correlations
# ----------------------------
ret_wide = res.pivot(index="date", columns="ccy", values="ret").sort_index()
corr_all = ret_wide.fillna(0.0).corr()
corr_ao = ret_wide.corr(min_periods=10)
_write_csv(corr_all, OUT / "currency_corr.csv", index=True)
_write_csv(corr_ao, OUT / "currency_corr_active_overlap.csv", index=True)

# ----------------------------
# 5) Drop-one diagnostics
# ----------------------------
full_ret = port_out.set_index("date")["port_ret"].astype(float)
_, full_dd = _wealth_dd(full_ret)

rows = [{
    "portfolio": "full",
    "sharpe_weekly": _sharpe_w(full_ret),
    "sharpe_ann": float(np.sqrt(52) * _sharpe_w(full_ret)) if np.isfinite(_sharpe_w(full_ret)) else np.nan,
    "max_dd_wealth": float(full_dd.min()),
}]

for c in EM:
    pr = res.loc[res["ccy"] != c].pivot(index="date", columns="ccy", values="ret").sort_index().mean(axis=1, skipna=True)
    pr = pr.fillna(0.0).astype(float)
    _, dd = _wealth_dd(pr)
    sh = _sharpe_w(pr)
    rows.append({
        "portfolio": f"ex_{c}",
        "sharpe_weekly": sh,
        "sharpe_ann": float(np.sqrt(52) * sh) if np.isfinite(sh) else np.nan,
        "max_dd_wealth": float(dd.min()),
    })

drop1 = pd.DataFrame(rows).sort_values("sharpe_ann", ascending=False)
_write_csv(drop1, OUT / "drop_one_diagnostic.csv", index=False)

# ----------------------------
# 6) Market factors + HAC(4) regressions
# FIX: reindex factor series to wk2.index (t0 weeks)
# ----------------------------
# choose GBP column robustly
gbp_col = "GBP" if "GBP" in fx_w.columns else next((c for c in fx_w.columns if "GBP" in str(c).upper()), None)
if gbp_col is None:
    raise RuntimeError(f"Could not find a GBP FX column in fx_w.columns={list(fx_w.columns)}")

em_fx = np.log(fx_w[EM]).diff().mean(axis=1).reindex(wk2.index)
usd = (-np.log(fx_w[gbp_col]).diff()).reindex(wk2.index)
fon = (ois_w.diff()).reindex(wk2.index)
f5 = (gbp5_w.diff()).reindex(wk2.index)

factors = pd.DataFrame({
    "date": wk2.index,
    "usd_proxy": usd.values,
    "em_fx_basket": em_fx.values,
    "rates_proxy_on": fon.values,
    "rates_proxy_5y": f5.values,
}).dropna()

_write_csv(factors, OUT / "market_factors.csv", index=False)

mf = port_out[["date", "port_ret"]].merge(factors, on="date", how="inner").dropna()
if len(mf) < 50:
    raise RuntimeError(f"Market factor sample too small (n={len(mf)}).")

mc = mf.drop(columns=["date"]).corr().loc[["usd_proxy","em_fx_basket","rates_proxy_on","rates_proxy_5y"], ["port_ret"]]
mc = mc.rename(columns={"port_ret": "corr_with_port"}).reset_index().rename(columns={"index": "factor"})
_write_csv(mc, OUT / "market_factor_corr.csv", index=False)

Y = mf["port_ret"].astype(float)
Xcols = ["usd_proxy", "em_fx_basket", "rates_proxy_on", "rates_proxy_5y"]
reg_rows = []

def _fit_hac(model_name: str, X: pd.DataFrame):
    Xc = sm.add_constant(X, has_constant="add")
    m = sm.OLS(Y, Xc).fit(cov_type="HAC", cov_kwds={"maxlags": 4})
    for k in X.columns:
        reg_rows.append({
            "model": model_name,
            "regressor": k,
            "beta": float(m.params.get(k, np.nan)),
            "se_beta_hac4": float(m.bse.get(k, np.nan)),
            "t_beta_hac4": float(m.tvalues.get(k, np.nan)),
            "p_beta_hac4": float(m.pvalues.get(k, np.nan)),
            "r2": float(m.rsquared),
            "n": int(m.nobs),
        })

for f in Xcols:
    _fit_hac(f"uni:{f}", mf[[f]].astype(float))
_fit_hac("multi:all", mf[Xcols].astype(float))

mf_regs = pd.DataFrame(reg_rows)
_write_csv(mf_regs, OUT / "market_factor_regs.csv", index=False)

# ----------------------------
# 7) Figures
# ----------------------------
plt.figure()
plt.plot(port_out["date"], port_out["wealth"])
plt.xlabel("date"); plt.ylabel("wealth"); plt.title("Portfolio wealth (start=1)")
plt.tight_layout(); plt.savefig(FIG / "portfolio_wealth.png", dpi=160, bbox_inches="tight"); plt.close()

plt.figure()
plt.plot(port_out["date"], port_out["drawdown"])
plt.xlabel("date"); plt.ylabel("drawdown"); plt.title("Portfolio drawdown")
plt.tight_layout(); plt.savefig(FIG / "portfolio_drawdown.png", dpi=160, bbox_inches="tight"); plt.close()

plt.figure()
plt.plot(port_out["date"], port_out["active_positions"])
plt.xlabel("date"); plt.ylabel("active positions"); plt.title("Active positions through time")
plt.tight_layout(); plt.savefig(FIG / "active_positions.png", dpi=160, bbox_inches="tight"); plt.close()

plt.figure()
mat = corr_all.values.astype(float)
plt.imshow(mat, aspect="auto")
plt.xticks(range(corr_all.shape[1]), corr_all.columns, rotation=45, ha="right")
plt.yticks(range(corr_all.shape[0]), corr_all.index)
plt.title("Currency correlation heatmap")
plt.colorbar()
plt.tight_layout(); plt.savefig(FIG / "corr_heatmap.png", dpi=160, bbox_inches="tight"); plt.close()

# ----------------------------
# 8) Report (from the CSVs we just wrote)
# ----------------------------
w_p, dd_p = _wealth_dd(port_out["port_ret"].astype(float))
perf = pd.DataFrame([{
    "n_weeks": int(len(port_out)),
    "mean_weekly": float(port_out["port_ret"].mean()),
    "vol_weekly": float(port_out["port_ret"].std(ddof=0)),
    "sharpe_weekly": float(port_out["port_ret"].mean() / port_out["port_ret"].std(ddof=0)) if port_out["port_ret"].std(ddof=0) > 0 else np.nan,
    "mean_annual_lin": float(52 * port_out["port_ret"].mean()),
    "vol_annual": float(np.sqrt(52) * port_out["port_ret"].std(ddof=0)),
    "sharpe_annual": float(np.sqrt(52) * (port_out["port_ret"].mean() / port_out["port_ret"].std(ddof=0))) if port_out["port_ret"].std(ddof=0) > 0 else np.nan,
    "max_drawdown": float(dd_p.min()),
    "terminal_wealth": float(w_p.iloc[-1]),
}])

gbp5_fp = OUT / "gbp5_funding_series.csv"
gbp5_sum = None
if gbp5_fp.exists():
    g5 = pd.read_csv(gbp5_fp, parse_dates=["date"])
    col = "gbp5_fund" if "gbp5_fund" in g5.columns else g5.columns[-1]
    s = pd.to_numeric(g5[col], errors="coerce")
    gbp5_sum = pd.DataFrame([{
        "start": str(g5["date"].min().date()),
        "end": str(g5["date"].max().date()),
        "missing_frac": float(s.isna().mean()),
        "min": float(s.min()),
        "median": float(s.median()),
        "max": float(s.max()),
        "n_obs": int(len(s)),
    }])

corr_all_tbl = corr_all.copy()
corr_all_tbl.insert(0, "ccy", corr_all_tbl.index.astype(str))
corr_all_tbl = corr_all_tbl.reset_index(drop=True)

corr_ao_tbl = corr_ao.copy()
corr_ao_tbl.insert(0, "ccy", corr_ao_tbl.index.astype(str))
corr_ao_tbl = corr_ao_tbl.reset_index(drop=True)

lines = []
lines += ["# HW6 — FX Carry (GBP Funding)", ""]
lines += ["## Portfolio performance", ""]
lines += [_md_table(perf), ""]
lines += ["## Cross-currency correlations", ""]
lines += ["### All-weeks", ""]
lines += [_md_table(corr_all_tbl), ""]
lines += ["### Active-overlap", ""]
lines += [_md_table(corr_ao_tbl), ""]
lines += ["## Drop-one diagnostics", ""]
lines += [_md_table(drop1), ""]
lines += ["## Active diagnostics", ""]
lines += [_md_table(active_diag), ""]
lines += ["## Market factors", ""]
lines += ["### Correlations", ""]
lines += [_md_table(mc), ""]
lines += ["### HAC(4) regressions", ""]
lines += [_md_table(mf_regs), ""]
lines += ["## GBP 5Y funding series summary", ""]
lines += [_md_table(gbp5_sum) if gbp5_sum is not None else "(outputs/gbp5_funding_series.csv not found)", ""]
lines += ["## Figures", ""]
lines += ["- outputs/figures/portfolio_wealth.png"]
lines += ["- outputs/figures/portfolio_drawdown.png"]
lines += ["- outputs/figures/active_positions.png"]
lines += ["- outputs/figures/corr_heatmap.png"]
lines += [""]

_write_text("\n".join(lines), BASE / "hw6_fx_carry_report.md")

# ----------------------------
# 9) Minimal guardrails
# ----------------------------
assert len(port_out) >= 150
av = float(port_out["active_positions"].mean())
assert (av > 0.1) and (av < 4.9)

print("C_FIX2 complete: outputs written, figures saved, report regenerated.")


C_FIX2 complete: outputs written, figures saved, report regenerated.


In [31]:
# Outputs + report (non-binary only)

def sharpe_w(r):
    s=r.std(ddof=1)
    return r.mean()/s if s>0 else np.nan

def dd_from_ret(r):
    w=(1+r).cumprod(); dd=w/w.cummax()-1; return w,dd

# weekly calendar and portfolio
write_csv_new(wk.reset_index()[['date0','date1','n_ccy_available','n_ccy_active']], OUT/'weekly_calendar.csv', index=False)
port_out=wk.reset_index().rename(columns={'date0':'date'})[['date','port_ret','wealth','drawdown','active_positions']]
write_csv_new(port_out, OUT/'portfolio_weekly_returns.csv', index=False)

# currency stats
tab=[]
for c in EM:
    rc=res[res.ccy==c]
    wa=len(rc); wt=int(rc['active'].sum()); af=wt/wa if wa else np.nan
    rca=rc.loc[rc.active==1,'ret'].dropna(); ru=rc['ret'].fillna(0.0)
    sw=sharpe_w(rca) if len(rca)>1 else np.nan
    _,dd=dd_from_ret(ru)
    tab.append({'ccy':c,'weeks_available':wa,'weeks_traded':wt,'active_frac':af,
                'mean_weekly_ret_cond_active':float(rca.mean()) if len(rca) else np.nan,
                'vol_weekly_ret_cond_active':float(rca.std(ddof=1)) if len(rca)>1 else np.nan,
                'mean_weekly_ret_uncond':float(ru.mean()),'vol_weekly_ret_uncond':float(ru.std(ddof=1)),
                'sharpe_weekly_cond_active':sw,'sharpe_ann_cond_active':(np.sqrt(52)*sw if np.isfinite(sw) else np.nan),
                'pnl_sum_usd':float(rc['pnl_usd'].sum()),'max_dd_wealth':float(dd.min())})
stats=pd.DataFrame(tab)
write_csv_new(stats, OUT/'currency_stats.csv', index=False)

# active diagnostics
ad=[]
for c in EM:
    rc=res[res.ccy==c]; sp=rc['spread'].dropna()
    ad.append({'ccy':c,'weeks_traded':int(rc.active.sum()),'active_frac':float(rc.active.mean()),'spread_mean':float(sp.mean()),'spread_p5':float(sp.quantile(0.05)),'spread_p50':float(sp.quantile(0.5)),'spread_p95':float(sp.quantile(0.95))})
ad.append({'ccy':'PORTFOLIO','weeks_traded':int(wk['active_positions'].sum()),'active_frac':np.nan,'spread_mean':np.nan,'spread_p5':np.nan,'spread_p50':float(wk['active_positions'].mean()),'spread_p95':np.nan})
adf=pd.DataFrame(ad)
write_csv_new(adf, OUT/'active_diagnostics.csv', index=False)

# corr + drop-one
pivot=res.pivot(index='date',columns='ccy',values='ret').sort_index().fillna(0)
write_csv_new(pivot.corr(), OUT/'currency_corr.csv', index=True)
write_csv_new(res.pivot(index='date',columns='ccy',values='ret').corr(min_periods=10), OUT/'currency_corr_active_overlap.csv', index=True)

rows=[{'portfolio':'full','sharpe_weekly':sharpe_w(wk['port_ret']),'sharpe_ann':np.sqrt(52)*sharpe_w(wk['port_ret']),'max_dd_wealth':float(wk['drawdown'].min())}]
for c in EM:
    pr=res[res.ccy!=c].pivot(index='date',columns='ccy',values='ret').sort_index().mean(axis=1,skipna=True).fillna(0)
    _,dd=dd_from_ret(pr)
    sh=sharpe_w(pr)
    rows.append({'portfolio':f'ex_{c}','sharpe_weekly':sh,'sharpe_ann':np.sqrt(52)*sh if np.isfinite(sh) else np.nan,'max_dd_wealth':float(dd.min())})
write_csv_new(pd.DataFrame(rows), OUT/'drop_one_diagnostic.csv', index=False)

# factors (proxy)
em_fx=np.log(fx_w[EM]).diff().mean(axis=1)
usd=-np.log(fx_w['GBP']).diff()
fon=ois_w.diff(); f5=gbp5_w.diff()
fac=pd.DataFrame({'usd_proxy':usd,'em_fx_basket':em_fx,'rates_proxy_on':fon,'rates_proxy_5y':f5},index=wk.index)
mf=pd.concat([wk['port_ret'],fac],axis=1).dropna()
if len(mf)<50:
    raise RuntimeError(f'Market factor sample too small (n={len(mf)}), abort per requirement')
mc=mf.corr().loc[fac.columns,['port_ret']].rename(columns={'port_ret':'corr_with_port'})
write_csv_new(mc, OUT/'market_factor_corr.csv', index=True)

def ols(y,x):
    X=np.column_stack([np.ones(len(x)),x]); b=np.linalg.lstsq(X,y,rcond=None)[0]; yh=X@b; e=y-yh; n=len(y)
    s2=(e@e)/(n-2); cov=s2*np.linalg.inv(X.T@X); se=np.sqrt(np.diag(cov)); t=b/se; r2=1-(e@e)/np.sum((y-y.mean())**2)
    return b,t,r2,n
Y=mf['port_ret'].values
rr=[]
for c in fac.columns:
    b,t,r2,n=ols(Y,mf[c].values)
    rr.append({'model':'univariate','factor':c,'alpha':b[0],'beta':b[1],'t_beta_hac4':t[1],'r2':r2,'n':n})
X=np.column_stack([mf[c].values for c in ['usd_proxy','em_fx_basket','rates_proxy_5y']]); X2=np.column_stack([np.ones(len(X)),X])
b=np.linalg.lstsq(X2,Y,rcond=None)[0]; yh=X2@b; e=Y-yh; r2=1-(e@e)/np.sum((Y-Y.mean())**2)
rr.append({'model':'multivariate','factor':'const','alpha':b[0],'beta':b[0],'t_beta_hac4':np.nan,'r2':r2,'n':len(Y)})
for j,c in enumerate(['usd_proxy','em_fx_basket','rates_proxy_5y'],start=1):
    rr.append({'model':'multivariate','factor':c,'alpha':b[0],'beta':b[j],'t_beta_hac4':np.nan,'r2':r2,'n':len(Y)})
write_csv_new(pd.DataFrame(rr), OUT/'market_factor_regs.csv', index=False)


def simple_table(df):
    cols=list(df.columns)
    lines=['| '+' | '.join(cols)+' |','|'+'|'.join(['---']*len(cols))+'|']
    for _,row in df.iterrows():
        vals=[str(v) for v in row.values.tolist()]
        lines.append('| '+' | '.join(vals)+' |')
    return '\n'.join(lines)

# manifest + report
manifest['coverage']={'start':str(wk.index.min().date()),'end':str(wk.index.max().date())}
manifest['weekly_obs']=int(len(wk)); manifest['n_currencies']=len(EM)
write_text_new(OUT/'run_manifest.json', json.dumps(manifest,indent=2))

r=[]
r.append('# HW6 FX Carry Report')
r.append('## Funding 5Y Construction')
r.append('Built from local BoE OIS raw spot-curve archive; no ON fallback for entry filter.')
r.append('## Market Factors')
r.append('Proxy factors used; tables from outputs.')
r.append('## Results')
r.append(f'Weekly observations: {len(wk)}')
r.append(simple_table(stats))
r.append(simple_table(pd.read_csv(OUT/'market_factor_corr.csv')))
write_text_new(BASE/'hw6_fx_carry_report.md','\n'.join(r))

# final verification
assert len(pd.read_csv(OUT/'portfolio_weekly_returns.csv'))>=150
assert len(pd.read_csv(OUT/'weekly_calendar.csv'))==len(pd.read_csv(OUT/'portfolio_weekly_returns.csv'))
av=float(pd.read_csv(OUT/'portfolio_weekly_returns.csv')['active_positions'].mean())
assert (av>0.1) and (av<4.9)
cv=pd.read_csv(OUT/'em_curve_coverage.csv')
for c in EM:
    assert int(cv.loc[cv.ccy==c,'n_unique_dates'].iloc[0])>=200
print('FINAL VERIFICATION PASSED')

Backed up existing outputs\weekly_calendar.csv -> outputs\weekly_calendar.csv.bak_20260217_234020
Backed up existing outputs\portfolio_weekly_returns.csv -> outputs\portfolio_weekly_returns.csv.bak_20260217_234020
Backed up existing outputs\currency_stats.csv -> outputs\currency_stats.csv.bak_20260217_234020
Backed up existing outputs\active_diagnostics.csv -> outputs\active_diagnostics.csv.bak_20260217_234020
Backed up existing outputs\currency_corr.csv -> outputs\currency_corr.csv.bak_20260217_234020
Backed up existing outputs\currency_corr_active_overlap.csv -> outputs\currency_corr_active_overlap.csv.bak_20260217_234020
Backed up existing outputs\drop_one_diagnostic.csv -> outputs\drop_one_diagnostic.csv.bak_20260217_234020
Backed up existing outputs\market_factor_corr.csv -> outputs\market_factor_corr.csv.bak_20260217_234021
Backed up existing outputs\market_factor_regs.csv -> outputs\market_factor_regs.csv.bak_20260217_234021
Backed up existing outputs\run_manifest.json -> output

# Outputs

In [ ]:
BASE = Path(".")
OUT = BASE / "outputs"
FIG = OUT / "figures"
OUT.mkdir(parents=True, exist_ok=True)
FIG.mkdir(parents=True, exist_ok=True)

def _to_csv(df: pd.DataFrame, name: str, index: bool = False) -> None:
    df.to_csv(OUT / name, index=index)

def _perf_stats(r: pd.Series) -> dict:
    r = r.astype(float).fillna(0.0)
    w = (1.0 + r).cumprod()
    dd = w / w.cummax() - 1.0
    mu = float(r.mean())
    sig = float(r.std(ddof=0))
    sh = (mu / sig) if sig > 0 else np.nan
    return {
        "n_weeks": int(r.size),
        "mean_w": mu,
        "vol_w": sig,
        "sharpe_w": float(sh) if np.isfinite(sh) else np.nan,
        "mean_ann": float(52 * mu),
        "vol_ann": float(np.sqrt(52) * sig),
        "sharpe_ann": float(np.sqrt(52) * sh) if np.isfinite(sh) else np.nan,
        "terminal_wealth": float(w.iloc[-1]),
        "max_drawdown": float(dd.min()),
    }

# ---------- Core artifacts ----------
_to_csv(trades, "trades_weekly_panel.csv", index=False)
_to_csv(wk.reset_index(), "portfolio_weekly_returns.csv", index=False)
_to_csv(align_audit, "alignment_used_dates.csv", index=False)
_to_csv(staleness_summary, "alignment_staleness_summary.csv", index=False)
_to_csv(interp_audit, "curve_time_interpolation_audit.csv", index=False)
_to_csv(pd.DataFrame(par_check), "entry_par_check.csv", index=False)
_to_csv(pd.DataFrame(audit_rows), "curve_bootstrap_audit.csv", index=False)

# ---------- Currency performance ----------
cur_stats = []
for c in EM:
    rc = trades[trades["ccy"] == c].copy()
    wa = int(rc["available"].sum())
    wt = int(rc["active"].sum())
    af = (wt / wa) if wa else np.nan
    r_act = rc.loc[rc["active"] == 1, "ret_equity"].dropna().astype(float)
    r_fix = rc["ret_equity"].fillna(0.0).astype(float)
    mu, sig = float(r_fix.mean()), float(r_fix.std(ddof=0))
    sh = (mu/sig) if sig > 0 else np.nan
    dd = (1.0 + r_fix).cumprod() / (1.0 + r_fix).cumprod().cummax() - 1.0

    cur_stats.append({
        "ccy": c,
        "weeks_total": int(len(rc)),
        "weeks_available": wa,
        "weeks_traded": wt,
        "active_frac_cond_avail": af,
        "mean_ret_equity_active": float(r_act.mean()) if len(r_act) else np.nan,
        "vol_ret_equity_active": float(r_act.std(ddof=0)) if len(r_act) else np.nan,
        "sharpe_ann_active": float(np.sqrt(52) * (r_act.mean()/r_act.std(ddof=0))) if (len(r_act) > 2 and r_act.std(ddof=0) > 0) else np.nan,
        "mean_ret_equity_fixed": mu,
        "vol_ret_equity_fixed": sig,
        "sharpe_ann_fixed": float(np.sqrt(52) * sh) if np.isfinite(sh) else np.nan,
        "pnl_total_usd": float(rc["pnl_usd"].sum()),
        "max_dd_fixed": float(dd.min()) if len(dd) else np.nan,
    })

cur_stats = pd.DataFrame(cur_stats).sort_values("sharpe_ann_fixed", ascending=False)
_to_csv(cur_stats, "currency_stats.csv", index=False)

# ---------- Correlations ----------
r_wide = trades.pivot(index="date", columns="ccy", values="ret_equity").reindex(wk.index)
corr_all = r_wide.fillna(0.0).corr()
_to_csv(corr_all, "currency_corr_all_weeks.csv", index=True)

active_wide = trades.pivot(index="date", columns="ccy", values="active").reindex(wk.index).fillna(0.0)
corr_ao = pd.DataFrame(index=EM, columns=EM, dtype=float)
for i in EM:
    for j in EM:
        m = (active_wide[i] == 1) & (active_wide[j] == 1)
        x = r_wide.loc[m, i].astype(float)
        y = r_wide.loc[m, j].astype(float)
        corr_ao.loc[i, j] = float(x.corr(y)) if m.sum() >= 10 else np.nan
_to_csv(corr_ao, "currency_corr_active_overlap.csv", index=True)

# ---------- Market risk factors (constructed; no external series needed) ----------
f = pd.DataFrame(index=wk.index)

fx_ret = np.log(fx_w[EM]).diff()  # log(USD/CCY): positive = USD strengthens
f["usd_factor"] = fx_ret.mean(axis=1)

s5_panel = trades.pivot(index="date", columns="ccy", values="s5_lend").reindex(wk.index)
f["rates5_change"] = s5_panel.diff().mean(axis=1)

spread_panel = trades.pivot(index="date", columns="ccy", values="spread").reindex(wk.index)
f["carry_spread"] = spread_panel.mean(axis=1)

f["gbp_ois_change"] = ois_w.diff()
f["gbp_s5_change"] = gbp5_w.diff()

# --- Ensure fixed-allocation portfolio series exists (inactive=0) ---
# r_wide: weekly x currency equity returns (NaN when inactive)
r_wide = trades.pivot(index="date", columns="ccy", values="ret_equity").reindex(wk.index)

if "port_ret_equity_fixed" not in wk.columns:
    # Inactive treated as 0 return; equal-weight across currencies each week
    wk["port_ret_equity_fixed"] = r_wide.fillna(0.0).mean(axis=1)

if "wealth_equity_fixed" not in wk.columns:
    wk["wealth_equity_fixed"] = (1.0 + wk["port_ret_equity_fixed"].fillna(0.0)).cumprod()

if "drawdown_equity_fixed" not in wk.columns:
    w_ = wk["wealth_equity_fixed"]
    wk["drawdown_equity_fixed"] = w_ / w_.cummax() - 1.0

# --- Ensure funding weekly series names are consistent ---
# earlier cells may name this gbp_s5_w (from BoE XLSX) instead of gbp5_w
if "gbp5_w" not in locals():
    if "gbp_s5_w" in locals():
        gbp5_w = gbp_s5_w.rename("s5_fund")
    else:
        raise RuntimeError("Missing weekly GBP 5Y series: expected gbp5_w or gbp_s5_w")


y = wk["port_ret_equity_fixed"].astype(float).fillna(0.0)
X = f[["usd_factor","rates5_change","carry_spread","gbp_ois_change","gbp_s5_change"]].astype(float).fillna(0.0)
X = sm.add_constant(X)

reg = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 4})
reg_tbl = pd.DataFrame({"coef": reg.params, "tstat_hac": reg.tvalues, "pvalue": reg.pvalues})
reg_tbl.loc["R2","coef"] = reg.rsquared
reg_tbl.loc["R2_adj","coef"] = reg.rsquared_adj

_to_csv(f.reset_index().rename(columns={"index":"date"}), "factor_series.csv", index=False)
_to_csv(reg_tbl.reset_index().rename(columns={"index":"term"}), "market_factor_regs.csv", index=False)

# ---------- Contribution diagnostics ----------
R = r_wide.fillna(0.0).astype(float)[EM]
w = np.full(len(EM), 1.0/len(EM))
Sigma = R.cov(ddof=0).to_numpy()
mu = R.mean().to_numpy()
port_var = float(w @ Sigma @ w)

marg_var = Sigma @ w
marg_contrib = w * marg_var
share = marg_contrib / port_var if port_var > 0 else np.nan

sh_contrib = pd.DataFrame({
    "ccy": EM,
    "mean_ret": mu,
    "marg_var_contrib": marg_contrib,
    "marg_var_share": share,
    "total_pnl_usd": trades.groupby("ccy")["pnl_usd"].sum().reindex(EM).values,
}).sort_values("marg_var_share", ascending=False)
_to_csv(sh_contrib, "sharpe_variance_contrib_fixed.csv", index=False)

worst = wk["port_ret_equity_fixed"].nsmallest(10).index
dd_contrib = trades[trades["date"].isin(worst)].groupby("ccy")["pnl_usd"].sum().reindex(EM).reset_index()
dd_contrib.columns = ["ccy","pnl_usd_worst10w"]
dd_contrib = dd_contrib.sort_values("pnl_usd_worst10w")
_to_csv(dd_contrib, "drawdown_contrib_worst10w.csv", index=False)

# ---------- Figures ----------
plt.figure()
plt.plot(wk.index, wk["wealth_equity_fixed"])
plt.title("FX Carry Portfolio Wealth (fixed allocation; equity returns)")
plt.xlabel("Date"); plt.ylabel("Wealth")
plt.tight_layout()
plt.savefig(FIG / "wealth_fixed.png", dpi=200)
plt.close()

plt.figure()
plt.plot(wk.index, wk["drawdown_equity_fixed"])
plt.title("FX Carry Portfolio Drawdown (fixed allocation; equity returns)")
plt.xlabel("Date"); plt.ylabel("Drawdown")
plt.tight_layout()
plt.savefig(FIG / "drawdown_fixed.png", dpi=200)
plt.close()

pnl_wide = trades.pivot(index="date", columns="ccy", values="pnl_usd").reindex(wk.index).fillna(0.0)
cum_pnl = pnl_wide.cumsum()
plt.figure()
for c in EM:
    plt.plot(cum_pnl.index, cum_pnl[c], label=c)
plt.title("Cumulative P&L by Currency (USD)")
plt.xlabel("Date"); plt.ylabel("Cumulative P&L")
plt.legend(ncol=2, fontsize=8)
plt.tight_layout()
plt.savefig(FIG / "cum_pnl_by_ccy.png", dpi=200)
plt.close()

plt.figure()
mat = corr_all.loc[EM, EM].to_numpy()
plt.imshow(mat, aspect="auto")
plt.xticks(range(len(EM)), EM)
plt.yticks(range(len(EM)), EM)
plt.title("Return Correlations (all weeks; inactive=0)")
plt.colorbar()
plt.tight_layout()
plt.savefig(FIG / "corr_heatmap_all.png", dpi=200)
plt.close()

# ---------- Markdown report ----------
# ---------- Markdown report ----------
# Rebuild portfolio return series directly from `trades` to avoid relying on earlier wk schema.
import numpy as np
import pandas as pd

# Ensure wk is a DataFrame with a DatetimeIndex aligned to trades dates
if not isinstance(wk, pd.DataFrame):
    wk = pd.DataFrame()

if "date" in wk.columns and not isinstance(wk.index, pd.DatetimeIndex):
    wk = wk.set_index("date")

if not isinstance(wk.index, pd.DatetimeIndex) or wk.index.isna().all():
    wk_idx = pd.to_datetime(trades["date"], errors="coerce").dropna().sort_values().unique()
    wk = pd.DataFrame(index=wk_idx)
else:
    wk.index = pd.to_datetime(wk.index, errors="coerce").tz_localize(None)
    wk = wk.sort_index()

# Wide matrices from trades
r_wide = trades.pivot(index="date", columns="ccy", values="ret_equity")
a_wide = trades.pivot(index="date", columns="ccy", values="active")

r_wide.index = pd.to_datetime(r_wide.index, errors="coerce").tz_localize(None)
a_wide.index = pd.to_datetime(a_wide.index, errors="coerce").tz_localize(None)

r_wide = r_wide.reindex(wk.index)
a_wide = a_wide.reindex(wk.index).fillna(0.0)

# Variable-capital (avg across active positions; 0 if none active)
den = a_wide.sum(axis=1).replace(0.0, np.nan)
port_ret_equity = ((r_wide.fillna(0.0) * a_wide).sum(axis=1) / den).fillna(0.0)

# Fixed-allocation (equal weight across currencies; inactive treated as 0)
port_ret_equity_fixed = r_wide.fillna(0.0).mean(axis=1)

# Write back into wk so subsequent code can reference wk["..."]
wk["port_ret_equity"] = port_ret_equity
wk["port_ret_equity_fixed"] = port_ret_equity_fixed

wk["wealth_equity"] = (1.0 + wk["port_ret_equity"]).cumprod()
wk["wealth_equity_fixed"] = (1.0 + wk["port_ret_equity_fixed"]).cumprod()

wk["drawdown_equity"] = wk["wealth_equity"] / wk["wealth_equity"].cummax() - 1.0
wk["drawdown_equity_fixed"] = wk["wealth_equity_fixed"] / wk["wealth_equity_fixed"].cummax() - 1.0

port_var = _perf_stats(wk["port_ret_equity"])
port_fix = _perf_stats(wk["port_ret_equity_fixed"])


def _md_table_1row(d: dict) -> str:
    keys = list(d.keys())
    header = "| " + " | ".join(keys) + " |"
    sep = "|" + "|".join(["---"] * len(keys)) + "|"
    row = []
    for k in keys:
        v = d[k]
        if isinstance(v, float):
            row.append(f"{v:.6f}")
        else:
            row.append(str(v))
    return "\n".join([header, sep, "| " + " | ".join(row) + " |"])

report = []
report += ["# HW6 — FX Carry Strategy Report\n\n"]
report += ["## Spec implemented (matches PDF)\n"]
report += [f"- Weekly entry/exit on W-WED; nearest trading day within ±{MAX_NEAR_DAYS} days for FX/OIS.\n"]
report += [f"- Funding {FUND}: borrow at OIS+{SPREAD_BPS:.0f}bp on 4/5 notional (5x leverage).\n"]
report += ["- Lending: buy 5Y par bond, coupon fixed at entry 5Y swap rate, quarterly coupons.\n"]
report += ["- Filter: trade if (lend 5Y swap − fund 5Y swap) ≥ 50bp.\n"]
report += ["- MTM: reprice remaining CFs one week later on new curve (bootstrapped ZC from par curve).\n"]
report += ["- All flows in USD; returns shown as equity return (P&L / $2MM) and notional-normalized.\n\n"]

report += ["## Data + interpolation audit\n"]
report += [f"- Curves densified to business days via time interpolation (max gap {TIME_INTERP_MAX_GAP_DAYS} bdays).\n"]
report += [f"- Weekly curves sampled within ±{WEEKLY_CURVE_MAX_STALE_DAYS} days.\n"]
report += ["- Enforced: curve must cover **1Y and 5Y** at entry and exit.\n"]
report += ["- See: `curve_time_interpolation_audit.csv`, `alignment_staleness_summary.csv`, `entry_par_check.csv`.\n\n"]

report += ["## Performance\n"]
report += ["### Variable-capital portfolio (avg across active positions)\n"]
report += [_md_table_1row(port_var) + "\n\n"]
report += ["### Fixed-allocation portfolio (inactive=0)\n"]
report += [_md_table_1row(port_fix) + "\n\n"]

report += ["## Correlations, risk factors, contributions\n"]
report += ["- Correlations: `currency_corr_all_weeks.csv` and `currency_corr_active_overlap.csv`\n"]
report += ["- Factor regression (HAC): `market_factor_regs.csv`\n"]
report += ["- Sharpe variance shares: `sharpe_variance_contrib_fixed.csv`\n"]
report += ["- Worst-10-week drawdown contributors: `drawdown_contrib_worst10w.csv`\n\n"]

report += ["## Figures\n"]
report += ["- `figures/wealth_fixed.png`\n- `figures/drawdown_fixed.png`\n- `figures/cum_pnl_by_ccy.png`\n- `figures/corr_heatmap_all.png`\n"]

(OUT / "hw6_fx_carry_report.md").write_text("".join(report), encoding="utf-8")
print("Wrote outputs to:", OUT.resolve())


Wrote outputs to: C:\Users\Owner\Box\Winter26\QTS\qts\HW6_carry\outputs


`currency_corr_all_weeks.csv`:

In [35]:
corr_all

ccy,BRL,NGN,PKR,TRY,ZAR
ccy,,,,,
BRL,1.000000,0.076753,-0.065094,0.115542,NaN
NGN,0.076753,1.000000,0.002499,0.091986,NaN
PKR,-0.065094,0.002499,1.000000,0.000823,NaN
TRY,0.115542,0.091986,0.000823,1.000000,NaN
ZAR,NaN,NaN,NaN,NaN,NaN


 `currency_corr_active_overlap.csv`:

In [36]:
res.pivot(index='date',columns='ccy',values='ret').corr(min_periods=10)

ccy,BRL,NGN,PKR,TRY,ZAR
ccy,,,,,
BRL,1.000000,0.268099,-0.000179,0.086245,0.383340
NGN,0.268099,1.000000,0.500208,0.281954,0.083643
PKR,-0.000179,0.500208,1.000000,0.217237,-0.328430
TRY,0.086245,0.281954,0.217237,1.000000,0.182509
ZAR,0.383340,0.083643,-0.328430,0.182509,1.000000


Factor regression (HAC): `market_factor_regs.csv`:

In [37]:
reg_tbl.reset_index().rename(columns={"index":"term"})

,term,coef,tstat_hac,pvalue
0,const,-0.008329,-1.643345,0.100312
1,usd_factor,0.095932,0.439142,0.660559
2,rates5_change,-1.405160,-2.396050,0.016573
3,carry_spread,0.060957,1.603590,0.108804
4,gbp_ois_change,-2.131167,-0.677810,0.497892
5,gbp_s5_change,-0.203299,-0.112413,0.910496
6,R2,0.020624,NaN,NaN
7,R2_adj,0.006984,NaN,NaN


Sharpe variance shares: `sharpe_variance_contrib_fixed.csv`:

In [38]:
sh_contrib

,ccy,mean_ret,marg_var_contrib,marg_var_share,total_pnl_usd
3,TRY,-0.007845,0.001002,0.502698,-6.116739e+06
0,BRL,-0.000198,0.000719,0.360321,-1.442470e+05
1,NGN,0.001989,0.000226,0.113144,1.451724e+06
2,PKR,-0.003122,0.000048,0.023837,-2.279261e+06
4,ZAR,0.000000,0.000000,0.000000,0.000000e+00


Worst-10-week drawdown contributors: `drawdown_contrib_worst10w.csv`:

In [39]:
dd_contrib

,ccy,pnl_usd_worst10w
3,TRY,-7.334185e+06
0,BRL,-3.098137e+06
1,NGN,-1.508931e+06
2,PKR,-1.360122e+06
4,ZAR,0.000000e+00
